# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime
from scipy.stats import linregress
from pprint import pprint

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "../output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)
# cities


600

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "Imperial"

# Build query URL
query_url = f"{url}appid={weather_api_key}&units={units}&q="
# pprint(query_url)

limit = 0
lat = []
lng = []
temp = []
date = []
humd = []
wind = []
cloud = []
city = []

# Make a request for each of the indices
for x in range(len(cities)):
    print(f"Making request number: {x} for ID: {cities[x]}")
    response = requests.get(query_url + cities[x]).json()

#     pprint(response)

    try:
        city.append(response['name'])
        lat.append(response['coord']['lat'])
        lng.append(response['coord']['lon'])
        date.append(response['dt'])
        temp.append(response['main']['temp_max'])
        humd.append(response['main']['humidity'])
        wind.append(response['wind']['speed'])
        cloud.append(response['clouds']['all'])
    except:
        print("value error pass")
       
    limit += 1
    if limit == 50: break
    time.sleep(1)  
    
    
    
    
    
    
    


Making request number: 0 for ID: hermanus
Making request number: 1 for ID: cabo san lucas
Making request number: 2 for ID: lorengau
Making request number: 3 for ID: yellowknife
Making request number: 4 for ID: sao joao da barra
Making request number: 5 for ID: hithadhoo
Making request number: 6 for ID: albany
Making request number: 7 for ID: bluff
Making request number: 8 for ID: santa maria
Making request number: 9 for ID: krasnyy chikoy
Making request number: 10 for ID: cherskiy
Making request number: 11 for ID: santa cruz
Making request number: 12 for ID: ushuaia
Making request number: 13 for ID: barrow
Making request number: 14 for ID: vaini
Making request number: 15 for ID: kamenka
Making request number: 16 for ID: butaritari
Making request number: 17 for ID: rikitea
Making request number: 18 for ID: tasiilaq
Making request number: 19 for ID: camargo
Making request number: 20 for ID: kushiro
Making request number: 21 for ID: wad madani
Making request number: 22 for ID: nikolskoye


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [4]:
# Export the city data into a .csv

# create a data frame from cities, lat, and temp
weather_df = dict({
    "city": city,
    "date": date,
    "lat": lat,
    "lng": lng,
    "temp": temp,
    "humd": humd,
    "wind": wind,
    "cloud": cloud
})

weather_data = pd.DataFrame.from_dict(weather_df)
# weather_data = weather_data.transpose()
weather_data.to_csv("Weather_Data.csv", index=False)
# Convert Unix Time Stamp to regular date
tmpdate = datetime.datetime.fromtimestamp(int(weather_data["date"][0])).strftime('%Y-%m-%d')
weather_data

,city,date,lat,lng,temp,humd,wind,cloud
0,Hermanus,1613067774,-34.4187,19.2345,75.00,80,1.99,0
1,Cabo San Lucas,1613067906,22.8909,-109.9124,75.00,60,3.44,20
2,Lorengau,1613067997,-2.0226,147.2712,80.92,77,7.87,100
3,Yellowknife,1613068072,62.4560,-114.3525,-4.00,84,11.50,90
4,São João da Barra,1613067786,-21.6403,-41.0511,86.00,58,20.71,20
5,Hithadhoo,1613068075,-0.6000,73.0833,81.41,80,13.09,16
6,Albany,1613068076,42.6001,-73.9662,30.00,41,3.00,94
7,Bluff,1613068077,-46.6000,168.3333,50.86,81,13.58,56
8,Santa Maria,1613067886,-29.6842,-53.8069,84.20,65,5.75,75
9,Krasnyy Chikoy,1613068079,50.3643,108.7512,-2.74,87,2.46,66


## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.

humidityover = weather_data.loc[weather_data['humd'] > 100]
print(humidityover)


# index = weather_data.index
# humidityover = weather_data["humd"] > 100
# humdindices = index[humidityover] get only rows with > 100
# indices_list = humdindices.tolist()
# print(indices_list)


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
# scatter plot
plt.scatter(weather_data["lat"],weather_data["temp"], marker="o", alpha = 0.75)

# labels and ttile
plt.title("City Latitude vs. Temperature (" + tmpDate + ")")
plt.xlabel("Latitude")
plt.ylabel("Temperature (F)")
plt.grid(True)

# Save the figure
plt.savefig("latandtemp.png")

# Show plot
plt.show()

## Latitude vs. Humidity Plot

In [ ]:
# scatter plot
plt.scatter(weather_data["lat"],weather_data["humd"], marker="o", alpha = 0.75)

# labels and title
plt.title("City Latitude vs. Humidity (" + tmpDate + ")")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.grid(True)

# Save the figure
plt.savefig("latandhumd.png")

# Show plot
plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:
# scatter plot
plt.scatter(weather_data["lat"],weather_data["cloud"], marker="o", alpha = 0.75)

# labels and title
plt.title("City Latitude vs. Cloudiness (" + tmpDate + ")")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.grid(True)

# Save the figure
plt.savefig("latandcloud.png")

# Show plot
plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:
# scatter plot
plt.scatter(weather_data["lat"],weather_data["wind"], marker="o", alpha = 0.75)

# labels and title
plt.title("City Latitude vs. Wind Speed (" + tmpDate + ")")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.grid(True)

# Save the figure
plt.savefig("latandwind.png")

# Show plot
plt.show()

## Linear Regression

In [ ]:
# northern hemisphre is north of equater, while southern hemishere is south of equator. sciencing.com(2018)
northern = weather_data.loc[weather_data["lat"] >= 0.0]
northern.reset_index(inplace=True)

southern = weather_data.loc[weather_data["lat"] < 0.0]
southern.reset_index(inplace=True)



####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
# assign x and y values
x_values = northern["lat"]
y_values = northern["temp"]

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.ylabel('Maximum Temperature')
plt.xlabel('Latitude')
plt.title("Northern Hemisphere - Max Temp vs. Latitude (" + tmpDate + ")")

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
## assign x and y values
x_values = southern["lat"]
y_values = southern["temp"]

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.ylabel('Maximum Temperature')
plt.xlabel('Latitude')
plt.title("Southern Hemisphere - Max Temp vs. Latitude (" + tmpDate + ")")

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
# assign x and y values
x_values = northern["lat"]
y_values = northern["humd"]

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.ylabel('Humidity (%)')
plt.xlabel('Latitude')
plt.title("Northern Hemisphere - Humidity (%) vs. Latitude (" + tmpDate + ")")

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
# assign x and y values
x_values = southern["lat"]
y_values = southern["humd"]

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot

plt.ylabel('Humidity')
plt.xlabel('Latitude')
plt.title("Southern Hemisphere - Maximum Humidity (%) vs. Latitude (" + tmpDate + ")")

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
# assign x and y values
x_values = northern["lat"]
y_values = northern["cloud"]

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.ylabel('Cloudiness')
plt.xlabel('Latitude')
plt.title("Northern Hemisphere - Cloudiness vs. Latitude (" + tmpDate + ")")

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
# assign x and y values
x_values = southern["lat"]
y_values = southern["cloud"]

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.ylabel('Cloudiness')
plt.xlabel('Latitude')
plt.title("Southern Hemisphere - Cloudiness vs. Latitude (" + tmpDate + ")")

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
# assign x and y values
x_values = northern["lat"]
y_values = northern["wind"]

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.ylabel('Wind Speed (mph)')
plt.xlabel('Latitude')
plt.title("Northern Hemisphere - Wind Speed (mph) vs. Latitude (" + tmpDate + ")")

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
# assign x and y values
x_values = southern["lat"]
y_values = southern["wind"]

# Run linear regression
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

# Plot scatter plot
plt.scatter(x_values,y_values)

# Plot regression line
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(6,10),fontsize=15,color="red")

# Label plot
plt.ylabel('Wind Speed (mph)')
plt.xlabel('Latitude')
plt.title("Southern Hemisphere - Wind Speed (mph) vs. Latitude (" + tmpDate + ")")

# Print r square value
print(f"R squared: {rvalue**2}")

# Show plot
plt.show()